# 构建你的第一个知识库(上)

## Notebook说明

这个Notebook将教你如何加载和分割文档，为构建RAG（检索增强生成）系统打下基础。

**学习目标：**
- 理解RAG的工作原理和必要性
- 掌握多种文档加载器的使用（PDF、网页等）
- 学会智能分割文档以优化检索效果
- 了解不同分割策略的适用场景

**前置要求：**
- 完成01_langchain_basics
- Python 3.8+
- 准备一些测试文档（PDF、网页链接等）

## RAG简介

### 为什么需要RAG？

RAG（Retrieval-Augmented Generation，检索增强生成）解决了LLM的一个核心问题：**如何让AI基于你自己的文档回答问题**。

**RAG的三步走策略：**
1. **准备知识库**：加载文档 → 切分成小块 → 转换为向量存储
2. **检索相关内容**：用户提问 → 检索最相关的文档片段
3. **生成答案**：将检索内容+问题发给LLM → 生成基于文档的答案

今天我们专注第一步：**文档加载与智能分割**

---



In [5]:
# 导入基础模块
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader, TextLoader
from langchain_text_splitters  import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    MarkdownTextSplitter
)

print("✅ 模块导入成功！")

✅ 模块导入成功！


---

## 第一部分：文档加载

### 为什么需要专门的加载器？

不同格式的文档有不同的特点：
- **PDF**：复杂格式和布局，需要专门解析
- **网页**：包含HTML标签，需要提取纯文本
- **Word/Markdown**：各有特定的结构标记

LangChain的Document Loaders提供统一接口，自动处理这些差异。

### 示例1：加载PDF文档

PDF是最常见的文档格式之一。让我们看看如何加载它。

In [8]:
# 加载本地PDF文件（请替换为你自己的PDF路径）
# 如果没有PDF文件，可以先跳过这个cell

pdf_path = "files/ddia.pdf"  # 替换为实际文件路径

try:
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    
    print(f"✅ 成功加载了 {len(documents)} 页文档")
    print(f"\n第一页内容预览：")
    print(documents[0].page_content[:300] + "...")
    print(f"\n第一页元数据：{documents[0].metadata}")
    
except Exception as e:
    print(f"⚠️ PDF加载失败（如果没有PDF文件可以跳过）：{e}")

✅ 成功加载了 613 页文档

第一页内容预览：
Martin Kleppmann
Designing 
Data-Intensive 
Applications
THE BIG IDEAS BEHIND RELIABLE, SCALABLE,  
AND MAINTAINABLE SYSTEMS...

第一页元数据：{'producer': 'Antenna House PDF Output Library 2.6.0 (Linux64)', 'creator': 'AH CSS Formatter V6.0 MR2 for Linux64 : 6.0.2.5372 (2012/05/16 18:26JST)', 'creationdate': '2017-03-01T18:01:20+00:00', 'author': 'Martin Kleppmann', 'moddate': '2017-03-01T13:07:54-05:00', 'title': 'Designing Data-Intensive Applications', 'trapped': '/False', 'source': 'files/ddia.pdf', 'total_pages': 613, 'page': 0, 'page_label': 'Cover'}


**PyPDFLoader的特点：**
- 自动按页拆分文档
- 保留页码等元数据
- 返回Document对象列表

**Document对象结构：**
- `page_content`：文本内容
- `metadata`：元数据（页码、来源等）

### 示例2：加载网页内容

在线文档也是重要的知识来源。WebBaseLoader可以抓取网页并提取文本。

In [9]:
# 加载LangChain官方文档的一个页面
web_url = "https://python.langchain.com/docs/introduction/"

try:
    loader = WebBaseLoader(web_url)
    documents = loader.load()
    
    print(f"✅ 成功加载网页")
    print(f"内容长度：{len(documents[0].page_content)} 字符")
    print(f"\n内容预览：")
    print(documents[0].page_content[:400] + "...")
    print(f"\n元数据：{documents[0].metadata}")
    
except Exception as e:
    print(f"⚠️ 网页加载失败：{e}")

✅ 成功加载网页
内容长度：3853 字符

内容预览：
LangChain overview - Docs by LangChainSkip to main content🚀 Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangChain + LangGraphSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangChain overviewLangChainLangGraphDeep AgentsIntegrationsLearnReferenceContributePythonOverviewChangelogGet startedInstallQuickstartPhilosophyCore compone...

元数据：{'source': 'https://python.langchain.com/docs/introduction/', 'title': 'LangChain overview - Docs by LangChain', 'language': 'en'}


**WebBaseLoader的能力：**
- 自动抓取HTML内容
- 使用BeautifulSoup解析并提取文本
- 移除脚本、样式等无关内容
- 保留文档结构

### 示例3：加载纯文本文件

In [ ]:
# 创建一个示例文本文件用于测试
sample_text = """LangChain是一个强大的框架，用于开发由语言模型驱动的应用程序。

它的核心优势包括：
1. 统一的模型接口
2. 丰富的组件库
3. 灵活的编排能力

通过LangChain，开发者可以快速构建RAG系统、Agent应用等复杂的AI解决方案。
"""

# 保存到文件
with open("files/sample.txt", "w", encoding="utf-8") as f:
    f.write(sample_text)

# 使用TextLoader加载
loader = TextLoader("files/sample.txt", encoding="utf-8")
documents = loader.load()

print(f"✅ 加载了 {len(documents)} 个文档")
print(f"\n内容：\n{documents[0].page_content}")

✅ 加载了 1 个文档

内容：
LangChain是一个强大的框架，用于开发由语言模型驱动的应用程序。

它的核心优势包括：
1. 统一的模型接口
2. 丰富的组件库
3. 灵活的编排能力

通过LangChain，开发者可以快速构建RAG系统、Agent应用等复杂的AI解决方案。



---

## 第二部分：文档分割

### 为什么必须分割文档？

**三大原因：**
1. **上下文窗口限制**：LLM有输入长度限制
2. **检索精度**：小块文本更容易精确匹配问题
3. **成本和延迟**：减少不必要的token消耗

**分割的核心挑战：**
- 适当的块大小（通常500-1500字符）
- 保持语义完整性（不在句子中间截断）
- 保留上下文（相邻块要有重叠）

### 示例1：CharacterTextSplitter（简单分割）

按固定字符数分割，不太考虑语义边界。

In [11]:
# 准备一段测试文本
long_text = """LangChain是一个开源框架，专门用于构建基于大型语言模型的应用程序。
它提供了一系列工具和抽象，使开发者能够更轻松地将LLM集成到实际应用中。

框架的核心组件包括：模型接口、提示词模板、链式调用、记忆管理和Agent系统。
通过这些组件的组合，开发者可以构建从简单的问答系统到复杂的自主Agent等各种应用。

LangChain支持多种LLM提供商，包括OpenAI、Anthropic、Hugging Face等。
这种灵活性使得开发者可以根据需求选择最合适的模型，而无需大幅修改代码。
""" * 3  # 重复3次以获得更长的文本

# 创建简单分割器
char_splitter = CharacterTextSplitter(
    chunk_size=200,      # 每块200字符
    chunk_overlap=50,    # 重叠50字符
    separator="\n"       # 优先在换行处分割
)

chunks = char_splitter.split_text(long_text)

print(f"分割成 {len(chunks)} 个文本块\n")
for i, chunk in enumerate(chunks[:3], 1):  # 显示前3个
    print(f"--- Chunk {i} (长度: {len(chunk)}) ---")
    print(chunk)
    print()

分割成 6 个文本块

--- Chunk 1 (长度: 157) ---
LangChain是一个开源框架，专门用于构建基于大型语言模型的应用程序。
它提供了一系列工具和抽象，使开发者能够更轻松地将LLM集成到实际应用中。
框架的核心组件包括：模型接口、提示词模板、链式调用、记忆管理和Agent系统。
通过这些组件的组合，开发者可以构建从简单的问答系统到复杂的自主Agent等各种应用。

--- Chunk 2 (长度: 170) ---
通过这些组件的组合，开发者可以构建从简单的问答系统到复杂的自主Agent等各种应用。
LangChain支持多种LLM提供商，包括OpenAI、Anthropic、Hugging Face等。
这种灵活性使得开发者可以根据需求选择最合适的模型，而无需大幅修改代码。
LangChain是一个开源框架，专门用于构建基于大型语言模型的应用程序。

--- Chunk 3 (长度: 157) ---
LangChain是一个开源框架，专门用于构建基于大型语言模型的应用程序。
它提供了一系列工具和抽象，使开发者能够更轻松地将LLM集成到实际应用中。
框架的核心组件包括：模型接口、提示词模板、链式调用、记忆管理和Agent系统。
通过这些组件的组合，开发者可以构建从简单的问答系统到复杂的自主Agent等各种应用。



### 示例2：RecursiveCharacterTextSplitter（智能分割）⭐

这是**最推荐**的分割器，会尝试在最合适的位置分割。

**分割优先级**：段落 > 句子 > 单词 > 字符

In [12]:
# 创建智能分割器
recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
    separators=["\n\n", "\n", "。", ".", " ", ""]  # 分割优先级
)

chunks = recursive_splitter.split_text(long_text)

print(f"分割成 {len(chunks)} 个文本块\n")
for i, chunk in enumerate(chunks[:3], 1):
    print(f"--- Chunk {i} (长度: {len(chunk)}) ---")
    print(chunk)
    print()

分割成 4 个文本块

--- Chunk 1 (长度: 158) ---
LangChain是一个开源框架，专门用于构建基于大型语言模型的应用程序。
它提供了一系列工具和抽象，使开发者能够更轻松地将LLM集成到实际应用中。

框架的核心组件包括：模型接口、提示词模板、链式调用、记忆管理和Agent系统。
通过这些组件的组合，开发者可以构建从简单的问答系统到复杂的自主Agent等各种应用。

--- Chunk 2 (长度: 248) ---
LangChain支持多种LLM提供商，包括OpenAI、Anthropic、Hugging Face等。
这种灵活性使得开发者可以根据需求选择最合适的模型，而无需大幅修改代码。
LangChain是一个开源框架，专门用于构建基于大型语言模型的应用程序。
它提供了一系列工具和抽象，使开发者能够更轻松地将LLM集成到实际应用中。

框架的核心组件包括：模型接口、提示词模板、链式调用、记忆管理和Agent系统。
通过这些组件的组合，开发者可以构建从简单的问答系统到复杂的自主Agent等各种应用。

--- Chunk 3 (长度: 248) ---
LangChain支持多种LLM提供商，包括OpenAI、Anthropic、Hugging Face等。
这种灵活性使得开发者可以根据需求选择最合适的模型，而无需大幅修改代码。
LangChain是一个开源框架，专门用于构建基于大型语言模型的应用程序。
它提供了一系列工具和抽象，使开发者能够更轻松地将LLM集成到实际应用中。

框架的核心组件包括：模型接口、提示词模板、链式调用、记忆管理和Agent系统。
通过这些组件的组合，开发者可以构建从简单的问答系统到复杂的自主Agent等各种应用。



### 对比：CharacterTextSplitter vs RecursiveCharacterTextSplitter

| 特性 | CharacterTextSplitter | RecursiveCharacterTextSplitter |
|------|----------------------|-------------------------------|
| 分割方式 | 按固定字符数 | 智能寻找最佳分割点 |
| 语义完整性 | ❌ 可能截断句子 | ✅ 保持语义完整 |
| 灵活性 | ⭐⭐ | ⭐⭐⭐⭐⭐ |
| 推荐程度 | 一般 | **强烈推荐** |

### 参数详解

让我们深入理解关键参数的影响。

In [13]:
# 测试不同的chunk_size
test_text = "这是一段测试文本。" * 50  # 生成较长文本

sizes = [100, 300, 500]

for size in sizes:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=size,
        chunk_overlap=50
    )
    chunks = splitter.split_text(test_text)
    print(f"chunk_size={size}: 分割成 {len(chunks)} 块")

chunk_size=100: 分割成 8 块
chunk_size=300: 分割成 2 块
chunk_size=500: 分割成 1 块


In [14]:
# 测试不同的chunk_overlap
overlaps = [0, 50, 100]

for overlap in overlaps:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=overlap
    )
    chunks = splitter.split_text(test_text)
    print(f"chunk_overlap={overlap}: 分割成 {len(chunks)} 块")

chunk_overlap=0: 分割成 2 块
chunk_overlap=50: 分割成 2 块
chunk_overlap=100: 分割成 2 块


---

## 完整实战：从加载到分割的端到端流程

让我们把所有知识整合到一个完整的示例中。

In [16]:
import pickle

print("=== 开始文档处理流程 ===")

# 步骤1：加载多个来源的文档
print("\n[步骤1] 正在加载文档...")

all_documents = []

# 1.1 加载网页（总是可以执行）
try:
    web_loader = WebBaseLoader([
        "https://python.langchain.com/docs/introduction/"
    ])
    web_docs = web_loader.load()
    all_documents.extend(web_docs)
    print(f"  ✅ 加载了 {len(web_docs)} 个网页文档")
except Exception as e:
    print(f"  ⚠️ 网页加载失败: {e}")

# 1.2 加载文本文件（使用之前创建的sample.txt）
try:
    text_loader = TextLoader("files/sample.txt", encoding="utf-8")
    text_docs = text_loader.load()
    all_documents.extend(text_docs)
    print(f"  ✅ 加载了 {len(text_docs)} 个文本文档")
except Exception as e:
    print(f"  ⚠️ 文本文件加载失败: {e}")

# 1.3 如果有PDF，也可以加载（可选）
# 取消下面的注释并替换路径
# try:
#     pdf_loader = PyPDFLoader("your_document.pdf")
#     pdf_docs = pdf_loader.load()
#     all_documents.extend(pdf_docs)
#     print(f"  ✅ 加载了 {len(pdf_docs)} 页PDF文档")
# except Exception as e:
#     print(f"  ⚠️ PDF加载失败: {e}")

print(f"\n总共加载了 {len(all_documents)} 个文档对象")

=== 开始文档处理流程 ===

[步骤1] 正在加载文档...
  ✅ 加载了 1 个网页文档
  ✅ 加载了 1 个文本文档

总共加载了 2 个文档对象


In [17]:
# 步骤2：配置智能分割器
print("\n[步骤2] 配置分割器...")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", "。", ".", " ", ""]
)

print("  ✅ 分割器配置完成")
print(f"     - chunk_size: 1000")
print(f"     - chunk_overlap: 200")


[步骤2] 配置分割器...
  ✅ 分割器配置完成
     - chunk_size: 1000
     - chunk_overlap: 200


In [18]:
# 步骤3：执行分割
print("\n[步骤3] 正在分割文档...")

chunks = text_splitter.split_documents(all_documents)

print(f"  ✅ 分割完成")


[步骤3] 正在分割文档...
  ✅ 分割完成


In [19]:
# 步骤4：分析分割结果
print("\n[步骤4] 分析分割结果...")
print(f"\n{'='*50}")
print(f"分割统计")
print(f"{'='*50}")
print(f"原始文档数量：{len(all_documents)}")
print(f"分割后块数量：{len(chunks)}")

chunk_lengths = [len(chunk.page_content) for chunk in chunks]
print(f"\n块大小统计：")
print(f"  平均大小：{sum(chunk_lengths)/len(chunk_lengths):.0f} 字符")
print(f"  最小大小：{min(chunk_lengths)} 字符")
print(f"  最大大小：{max(chunk_lengths)} 字符")


[步骤4] 分析分割结果...

分割统计
原始文档数量：2
分割后块数量：7

块大小统计：
  平均大小：587 字符
  最小大小：125 字符
  最大大小：986 字符


In [20]:
# 步骤5：查看部分结果
print("\n[步骤5] 查看部分分割结果...\n")

for i, chunk in enumerate(chunks[:3], 1):
    print(f"--- Chunk {i} ---")
    print(f"长度：{len(chunk.page_content)} 字符")
    print(f"来源：{chunk.metadata.get('source', '未知')}")
    print(f"内容预览：{chunk.page_content[:200]}...")
    print()


[步骤5] 查看部分分割结果...

--- Chunk 1 ---
长度：986 字符
来源：https://python.langchain.com/docs/introduction/
内容预览：LangChain overview - Docs by LangChainSkip to main content🚀 Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangChain + LangGraphSearch...⌘KAsk AIGitHubT...

--- Chunk 2 ---
长度：280 字符
来源：https://python.langchain.com/docs/introduction/
内容预览：.x is now available!For a complete list of changes and instructions on how to upgrade your code, see the release notes and migration guide.If you encounter any issues or have feedback, please open an ...

--- Chunk 3 ---
长度：942 字符
来源：https://python.langchain.com/docs/introduction/
内容预览：LangChain is the easiest way to start building agents and applications powered by LLMs. With under 10 lines of code, you can connect to OpenAI, Anthropic, Google, and more. LangChain provides a pre-bu...



In [21]:
# 步骤6：保存处理结果
print("[步骤6] 保存处理结果...")

with open("files/processed_chunks.pkl", "wb") as f:
    pickle.dump(chunks, f)

print("  ✅ 已保存到 processed_chunks.pkl")
print("\n=== 文档处理流程完成 ===")

[步骤6] 保存处理结果...
  ✅ 已保存到 processed_chunks.pkl

=== 文档处理流程完成 ===


---

## 最佳实践

### 不同文档类型的特殊处理

#### 对于代码文档

代码有特殊的结构（类、函数等），需要相应的分割策略。

In [22]:
# 针对Python代码的分割器
code_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    separators=["\n\nclass ", "\n\ndef ", "\n\n", "\n", " "]  # 优先在类和函数边界分割
)

sample_code = """
class LangChainHelper:
    def __init__(self):
        self.model = ChatOpenAI()
    
    def process(self, text):
        return self.model.invoke(text)

def load_documents(path):
    loader = PyPDFLoader(path)
    return loader.load()
""" * 5

code_chunks = code_splitter.split_text(sample_code)
print(f"代码分割成 {len(code_chunks)} 块")

代码分割成 1 块


#### 对于Markdown文档

In [23]:
# 针对Markdown的分割器（按标题层级）
md_splitter = MarkdownTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

sample_markdown = """
# LangChain教程

## 第一章：基础概念

LangChain是一个强大的框架...

### 1.1 核心组件

包括Models、Prompts、Chains等...

## 第二章：实战应用

让我们构建第一个应用...
""" * 3

md_chunks = md_splitter.split_text(sample_markdown)
print(f"Markdown分割成 {len(md_chunks)} 块")

Markdown分割成 1 块


---

## 扩展练习

### 练习1：实验不同的参数

尝试不同的chunk_size和chunk_overlap组合，观察结果的变化。

In [ ]:
# 在这里编写你的代码
# 提示：创建多个分割器，比较它们的输出

### 练习2：处理自己的文档

加载你自己的文档（PDF、网页或文本），并进行分割处理。

In [ ]:
# 在这里编写你的代码

### 练习3：分析块的质量

编写代码分析分割后的chunks：
- 统计每个chunk的长度分布
- 检查是否有chunks被截断在句子中间
- 验证相邻chunks之间的重叠

In [ ]:
# 在这里编写你的代码
# 提示：可以使用matplotlib绘制长度分布图

---

## 常见问题和解决方案

### Q1: 分割后丢失了重要的上下文怎么办？
**解决方案：**
- 增加chunk_overlap（如从100增加到300）
- 在每个chunk中添加文档元数据（标题、章节号等）
- 使用更大的chunk_size

### Q2: 某些chunks太短或太长？
**解决方案：**
- 调整separators的优先级
- 使用RecursiveCharacterTextSplitter而非CharacterTextSplitter
- 设置min_chunk_size参数（如果支持）

### Q3: PDF表格和图片信息丢失？
**解决方案：**
- 使用更高级的PDF解析器（如UnstructuredPDFLoader）
- 结合OCR技术提取图片文字
- 对表格数据使用专门的处理逻辑

---

## 下一步学习建议

🎯 **继续深入：**
- 学习Embedding（向量化）技术
- 了解向量数据库（Chroma、FAISS等）
- 实现完整的RAG检索系统

📚 下一篇教程：《构建你的第一个知识库（下）：向量化与高效检索》

---

## 总结

恭喜你完成了文档加载与分割的学习！你已经掌握了：

✅ **文档加载**
- 使用PyPDFLoader、WebBaseLoader等加载不同格式
- 理解Document对象的结构（content + metadata）
- 批量加载多个来源的文档

✅ **文档分割**
- RecursiveCharacterTextSplitter的智能分割策略
- chunk_size和chunk_overlap的平衡艺术
- 针对不同文档类型的特殊处理

✅ **最佳实践**
- 端到端的文档处理流程
- 常见问题的解决方案
- 分割质量的分析方法

**关键要点：**
- 分割是RAG系统的基础，直接影响检索质量
- 没有完美的分割策略，需要根据具体场景调整
- 保留足够的上下文（overlap）至关重要

下一步，我们将学习如何将这些文本块转换为向量，并实现高效的语义检索！🚀